In [1]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib
import sqlite3

import numpy as np
import pandas as pd
import dask.dataframe as dd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [2]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-TEI_XML_NLP'

### PART II: Extracting information from XML data files

#### Reading a single XML Data file to test:

In [3]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [4]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [5]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x0000022E89D0F4F8>


In [6]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [7]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [8]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [9]:
#for elem in root.iter():
for elem in root.iter('{http://www.tei-c.org/ns/1.0}entry'):
    for attrName, attrValue in elem.items():
        print(attrName + ' ==> ' + attrValue)
    #print(elem.tag)
    #print(elem.text)
    #print(elem.attrib)
    #print(elem.items())
    #print('\n')

{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e2
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e43
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e70
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e91
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e110
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e131
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e152
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e179
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e209
{http://www.w3.org/XML/1998/namespace}lang ==> b

{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18519
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18546
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18569
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18588
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18609
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18630
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18651
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e18672
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace

{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24764
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24785
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24804
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24827
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24851
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24870
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24893
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24916
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e24946
{http://www.w3.org/XML/19

{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27450
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27484
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27518
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27552
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27576
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27599
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27622
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e27643
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace

{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e44934
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e44957
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e44980
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e45005
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e45028
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e45052
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e45075
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace}id ==> d157_qdb-d1e45096
{http://www.w3.org/XML/1998/namespace}lang ==> bar
{http://www.w3.org/XML/1998/namespace

#### Iterating through all XML Data files

In [10]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_d-tei-02',
 'bel_e-tei-02',
 'bel_f-tei-02',
 'bel_g-tei-02',
 'bel_h-tei-02',
 'bel_i-j-tei-02',
 'bel_k-tei-02',
 'bel_l-tei-02',
 'bel_m-tei-02',
 'bel_n-o-p-tei-02',
 'bel_q-r-tei-02',
 'bel_s-tei-02',
 'bel_u-w-tei-02',
 'bel_z-tei-02']

In [11]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml
2 -- ./TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml
3 -- ./TEI-XML-2018/bel_d-tei-02/d158_qdb-TEI-02.xml
4 -- ./TEI-XML-2018/bel_d-tei-02/d159_qdb-TEI-02.xml
5 -- ./TEI-XML-2018/bel_d-tei-02/d160_qdb-TEI-02.xml
6 -- ./TEI-XML-2018/bel_d-tei-02/d161_qdb-TEI-02.xml
7 -- ./TEI-XML-2018/bel_d-tei-02/d162_qdb-TEI-02.xml
8 -- ./TEI-XML-2018/bel_d-tei-02/d163_qdb-TEI-02.xml
9 -- ./TEI-XML-2018/bel_d-tei-02/d164_qdb-TEI-02.xml
10 -- ./TEI-XML-2018/bel_d-tei-02/d165_qdb-TEI-02.xml
11 -- ./TEI-XML-2018/bel_d-tei-02/d166_qdb-TEI-02.xml
12 -- ./TEI-XML-2018/bel_d-tei-02/d167_qdb-TEI-02.xml
13 -- ./TEI-XML-2018/bel_d-tei-02/d168_qdb-TEI-02.xml
14 -- ./TEI-XML-2018/bel_d-tei-02/d169_qdb-TEI-02.xml
15 -- ./TEI-XML-2018/bel_d-tei-02/d170_qdb-TEI-02.xml
16 -- ./TEI-XML-2018/bel_d-tei-02/d171_qdb-TEI-02.xml
17 -- ./TEI-XML-2018/bel_d-tei-02/d172_qdb-TEI-02.xml
18 -- ./TEI-XML-2018/bel_d-tei-02/d173_qdb-TEI-02.xml
19 -- ./TEI-

311 -- ./TEI-XML-2018/bel_k-tei-02/k467_qdb-TEI-02.xml
312 -- ./TEI-XML-2018/bel_k-tei-02/k468_qdb-TEI-02.xml
313 -- ./TEI-XML-2018/bel_k-tei-02/k469_qdb-TEI-02.xml
314 -- ./TEI-XML-2018/bel_k-tei-02/k470_qdb-TEI-02.xml
315 -- ./TEI-XML-2018/bel_k-tei-02/k471_qdb-TEI-02.xml
316 -- ./TEI-XML-2018/bel_k-tei-02/k472_qdb-TEI-02.xml
317 -- ./TEI-XML-2018/bel_k-tei-02/k473_qdb-TEI-02.xml
318 -- ./TEI-XML-2018/bel_k-tei-02/k474_qdb-TEI-02.xml
319 -- ./TEI-XML-2018/bel_k-tei-02/k475_qdb-TEI-02.xml
320 -- ./TEI-XML-2018/bel_k-tei-02/k476_qdb-TEI-02.xml
321 -- ./TEI-XML-2018/bel_k-tei-02/k477_qdb-TEI-02.xml
322 -- ./TEI-XML-2018/bel_k-tei-02/k478_qdb-TEI-02.xml
323 -- ./TEI-XML-2018/bel_k-tei-02/k479_qdb-TEI-02.xml
324 -- ./TEI-XML-2018/bel_k-tei-02/k480_qdb-TEI-02.xml
325 -- ./TEI-XML-2018/bel_k-tei-02/k481_qdb-TEI-02.xml
326 -- ./TEI-XML-2018/bel_k-tei-02/k482_qdb-TEI-02.xml
327 -- ./TEI-XML-2018/bel_k-tei-02/k483_qdb-TEI-02.xml
328 -- ./TEI-XML-2018/bel_k-tei-02/k484_qdb-TEI-02.xml
329 -- ./T

In [12]:
t0 = time.time()

columns=['filename',
         'entry_id',
         'date',
         'usg_corresp', 
         'simple_placeName',
         'orig_placeName',
         'placeName_id',
         'Bundesland',
         'Bundesland_idno',
         'Großregion',
         'Großregion_idno',
         'Kleinregion',
         'Kleinregion_idno',
         'Gemeinde',
         'Gemeinde_idno',
         'Ort',
         'Ort_idno',
         'hauptlemma_unique',
         'hauptlemma_orig',
         'hauptlemma_normalized',
         'nebenlemma_unique',
         'nebenlemma_orig',
         'nebenlemma_normalized',
         'verweise',
         'archiv',
         'quelle',
         'quelleBearbeitet',
         'seite',
         'fragebogenNummer',
         'paragraph',
         'bibl',
         'tustep',
         'pos',
         'etym',
         'note_notabene',
         'note_anmerkung',
         'note_diverse',
         'cit_type',
         'cit_quote',
         'cit_def',
         'cit_interp',
         'cit_ref',
         'cit_pRef',
         'cit_note',
         'cit_re',
         'certainty',
        ]

#df_lemmas = pd.DataFrame(columns=columns)
df_lemmas = pd.DataFrame()
#dd_lemmas = dd.DataFrame(npartitions=3)
list_records = []

for fileid, teixmlfile in enumerate(onlyfiles):
    #tree = ET.parse(teixmlfile)
    #root = tree.getroot()
    root = ET.parse(teixmlfile).getroot()
    for entry in root.iter('{http://www.tei-c.org/ns/1.0}entry'):

        record = {'filename': teixmlfile,
                  'entry_id': None,
                  'date': None,
                  'usg_corresp': None,
                  'simple_placeName': None,
                  'orig_placeName': None,
                  'placeName_id': None,
                  'Bundesland': None,
                  'Bundesland_idno': None,
                  'Großregion': None,
                  'Großregion_idno': None,
                  'Kleinregion': None,
                  'Kleinregion_idno': None,
                  'Gemeinde': None,
                  'Gemeinde_idno': None,
                  'Ort': None,
                  'Ort_idno': None,
                  'hauptlemma_unique': None,
                  'hauptlemma_orig': None,
                  'hauptlemma_normalized': None,
                  'nebenlemma_unique': None,
                  'nebenlemma_orig': None,
                  'nebenlemma_normalized': None,
                  'verweise': None,
                  'archiv': None,
                  'quelle': None,
                  'quelleBearbeitet': None,
                  'seite': None,
                  'fragebogenNummer': None,
                  'paragraph': None,
                  'bibl': None,
                  'tustep': [],
                  'pos': None,
                  'etym': [],
                  'note_notabene': None,
                  'note_anmerkung': None,
                  'note_diverse': None,
                  'cit_type': None,
                  'cit_quote': [],
                  'cit_def': [],
                  'cit_interp': [],
                  'cit_ref': [],
                  'cit_pRef': [],
                  'cit_note': [],
                  'cit_re':[],
                  'certainty':[],
                 }

        record['entry_id'] = entry.attrib['{http://www.w3.org/XML/1998/namespace}id']
        
        for child in entry.iter():
            level = 'simple_placeName'
            if child.tag == '{http://www.tei-c.org/ns/1.0}usg':
                if 'corresp' in child.attrib:
                    record['usg_corresp'] = child.attrib['corresp']
                for subchild in child.iter():
                    if 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}placeName'\
                            and subchild.attrib['type'] == 'orig':
                        record['orig_placeName'] = subchild.text

                    elif 'ref' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}listPlace':
                        record['placeName_id'] = subchild.attrib['ref']

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Bundesland':
                        level = 'Bundesland'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Großregion':
                        level = 'Großregion'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Kleinregion':
                        level = 'Kleinregion'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Gemeinde':
                        level = 'Gemeinde'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Ort':
                        level = 'Ort'

                    if 'type' not in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}placeName':
                        record[level] = subchild.text
                    elif 'type' in child.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}idno':
                        record['{}_idno'.format(level)] = subchild.text

            elif child.tag == '{http://www.tei-c.org/ns/1.0}form' \
                        and 'type' in child.attrib:
                if child.attrib['type'] == 'hauptlemma':
                    for subchild in child.iter():
                        if subchild.tag == '{http://www.tei-c.org/ns/1.0}orth':
                            if 'type' not in subchild.attrib:
                                record['hauptlemma_unique'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'orig':
                                record['hauptlemma_orig'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'normalized':
                                record['hauptlemma_normalized'] = subchild.text
                elif child.attrib['type'] == 'nebenlemma':
                    for subchild in child.iter():
                        if subchild.tag == '{http://www.tei-c.org/ns/1.0}orth':
                            if 'type' not in subchild.attrib:
                                record['nebenlemma_unique'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'orig':
                                record['nebenlemma_orig'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'normalized':
                                record['nebenlemma_normalized'] = subchild.text       
                elif child.attrib['type'] == 'lautung':
                    for subchild in child.iter():
                            if 'notation' in subchild.attrib and subchild.attrib['notation'] == 'tustep':
                                record['tustep'].append(subchild.text)

            elif child.tag == '{http://www.tei-c.org/ns/1.0}pos':
                record['pos'] = child.text
            
            elif child.tag == '{http://www.tei-c.org/ns/1.0}date':
                record['date'] = child.text
                
            elif child.tag == '{http://www.tei-c.org/ns/1.0}note':
                if 'type' in child.attrib and child.attrib['type'] == 'notabene':
                    record['note_notabene'] = subchild.text
                elif 'type' in child.attrib and child.attrib['type'] == 'anmerkung':
                    record['note_anmerkung'] = child.text
                elif 'type' in child.attrib and child.attrib['type'] == 'diverse':
                    record['note_diverse'] = child.text
                elif 'type' in child.attrib:
                    print('new note type ', child.attrib['type'] )

            elif child.tag == '{http://www.tei-c.org/ns/1.0}cit':
                for subchild in child.iter():
                    if subchild.tag == '{http://www.tei-c.org/ns/1.0}cit':
                        record['cit_type'] = subchild.attrib['type']
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}quote':
                        record['cit_quote'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}def':
                        record['cit_def'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}interp':
                        record['cit_interp'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}ref':
                        record['cit_ref'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}pRef':
                        record['cit_pRef'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}note':
                        record['cit_note'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}re':
                        record['cit_re'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}certainty':
                        record['certainty'].append(subchild.text)
                    else:
                        print('new cit tag ', subchild.tag)                
                    
            elif child.tag == '{http://www.tei-c.org/ns/1.0}ref':
                if child.attrib['type'] == 'verweise':
                    record['verweise'] = child.text
                elif child.attrib['type'] == 'archiv':
                    record['archiv'] = child.text
                elif child.attrib['type'] == 'quelle':
                    record['quelle'] = child.text
                elif child.attrib['type'] == 'quelleBearbeitet':
                    record['quelleBearbeitet'] = child.text
                elif child.attrib['type'] == 'seite':
                    record['seite'] = child.text
                elif child.attrib['type'] == 'fragebogenNummer':
                    record['fragebogenNummer'] = child.text
                elif child.attrib['type'] == 'paragraph':
                    record['paragraph'] = child.text
                elif child.attrib['type'] == 'bibl':
                    record['bibl'] = child.text
                else:
                    print('new ref type --> ', child.attrib['type'])
            
            elif child.tag == '{http://www.tei-c.org/ns/1.0}etym':
                for subchild in child.iter():
                    record['etym'].append(subchild.text)
                    
        list_records.append(record)
    print('Processing time is {0:.2f} minutes for file {1}'.format(((time.time() - t0) / 60), fileid))

Processing time is 0.00 minutes for file 0
Processing time is 0.00 minutes for file 1
Processing time is 0.02 minutes for file 2
Processing time is 0.02 minutes for file 3
Processing time is 0.03 minutes for file 4
Processing time is 0.04 minutes for file 5
Processing time is 0.04 minutes for file 6
Processing time is 0.04 minutes for file 7
Processing time is 0.05 minutes for file 8
Processing time is 0.06 minutes for file 9
Processing time is 0.06 minutes for file 10
Processing time is 0.08 minutes for file 11
Processing time is 0.09 minutes for file 12
Processing time is 0.09 minutes for file 13
Processing time is 0.09 minutes for file 14
Processing time is 0.11 minutes for file 15
Processing time is 0.11 minutes for file 16
Processing time is 0.12 minutes for file 17
Processing time is 0.13 minutes for file 18
Processing time is 0.13 minutes for file 19
Processing time is 0.14 minutes for file 20
Processing time is 0.15 minutes for file 21
Processing time is 0.15 minutes for file 2

Processing time is 1.45 minutes for file 185
Processing time is 1.45 minutes for file 186
Processing time is 1.46 minutes for file 187
Processing time is 1.46 minutes for file 188
Processing time is 1.47 minutes for file 189
Processing time is 1.47 minutes for file 190
Processing time is 1.51 minutes for file 191
Processing time is 1.51 minutes for file 192
Processing time is 1.51 minutes for file 193
Processing time is 1.52 minutes for file 194
Processing time is 1.53 minutes for file 195
Processing time is 1.53 minutes for file 196
Processing time is 1.53 minutes for file 197
Processing time is 1.54 minutes for file 198
Processing time is 1.54 minutes for file 199
Processing time is 1.58 minutes for file 200
Processing time is 1.58 minutes for file 201
Processing time is 1.59 minutes for file 202
Processing time is 1.59 minutes for file 203
Processing time is 1.60 minutes for file 204
Processing time is 1.60 minutes for file 205
Processing time is 1.61 minutes for file 206
Processing

Processing time is 3.00 minutes for file 368
Processing time is 3.01 minutes for file 369
Processing time is 3.01 minutes for file 370
Processing time is 3.01 minutes for file 371
Processing time is 3.02 minutes for file 372
Processing time is 3.02 minutes for file 373
Processing time is 3.03 minutes for file 374
Processing time is 3.03 minutes for file 375
Processing time is 3.04 minutes for file 376
Processing time is 3.04 minutes for file 377
Processing time is 3.05 minutes for file 378
Processing time is 3.05 minutes for file 379
Processing time is 3.12 minutes for file 380
Processing time is 3.13 minutes for file 381
Processing time is 3.13 minutes for file 382
Processing time is 3.14 minutes for file 383
Processing time is 3.14 minutes for file 384
Processing time is 3.15 minutes for file 385
Processing time is 3.15 minutes for file 386
Processing time is 3.16 minutes for file 387
Processing time is 3.16 minutes for file 388
Processing time is 3.17 minutes for file 389
Processing

Processing time is 5.30 minutes for file 551
Processing time is 5.30 minutes for file 552
Processing time is 5.31 minutes for file 553
Processing time is 5.32 minutes for file 554
Processing time is 5.32 minutes for file 555
Processing time is 5.33 minutes for file 556
Processing time is 5.33 minutes for file 557
Processing time is 5.34 minutes for file 558
Processing time is 5.34 minutes for file 559
Processing time is 5.35 minutes for file 560
Processing time is 5.36 minutes for file 561
Processing time is 5.46 minutes for file 562
Processing time is 5.46 minutes for file 563
Processing time is 5.47 minutes for file 564
Processing time is 5.47 minutes for file 565
Processing time is 5.48 minutes for file 566
Processing time is 5.49 minutes for file 567
Processing time is 5.50 minutes for file 568
Processing time is 5.50 minutes for file 569
Processing time is 5.51 minutes for file 570
Processing time is 5.51 minutes for file 571
Processing time is 5.52 minutes for file 572
Processing

In [13]:
len(list_records)

2416499

In [16]:
with open('./list_lemmas.pkl', 'wb') as f:
    pickle.dump(list_records, f)

In [14]:
df_lemmas = pd.DataFrame(list_records)

In [15]:
df_lemmas.to_pickle('./df_lemmas.pkl')

In [2]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [3]:
df_lemmas.info(verbose=True, memory_usage='deep') #, max_cols=100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2416499 entries, 0 to 2416498
Data columns (total 46 columns):
Bundesland               object
Bundesland_idno          object
Gemeinde                 object
Gemeinde_idno            object
Großregion               object
Großregion_idno          object
Kleinregion              object
Kleinregion_idno         object
Ort                      object
Ort_idno                 object
archiv                   object
bibl                     object
certainty                object
cit_def                  object
cit_interp               object
cit_note                 object
cit_pRef                 object
cit_quote                object
cit_re                   object
cit_ref                  object
cit_type                 object
date                     object
entry_id                 object
etym                     object
filename                 object
fragebogenNummer         object
hauptlemma_normalized    object
hauptlemma_orig        

In [28]:
df_lemmas.head(10)

,Bundesland,Bundesland_idno,Gemeinde,Gemeinde_idno,Großregion,Großregion_idno,Kleinregion,Kleinregion_idno,Ort,Ort_idno,...,paragraph,placeName_id,pos,quelle,quelleBearbeitet,seite,simple_placeName,tustep,usg_corresp,verweise
0,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Egger Gl.,"{1B,1C} hist. TirWeistGl.",30,None,[diaun],None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Schneider Tir.(1965),"{1B,1C:X} Etym.^@ SCHNEIDER· (1963)",601,Chur,[diaun],None,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Tir.Wb.,"{1B,1C} Tir. TirWb.(1955-1956)","1,1293,337, f337^#3.4 = f.",Chur,[diaun],None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,[tib],None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,[tib],None,None
5,None,None,None,None,None,None,None,None,None,None,...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,"[tibo, tibå]",None,None
6,St.,3,Lobmingtal,None,sbair.ObSt.,3.1,mittl.obMurt.,3.1d,Kleinlobming,3.1d03,...,None,sigle:3.1d03,Interj,Gr.Lobming Geosich,{3.1d03} mittl.obMurt.:obMurt.:mObMurgeb.:ObMu...,None,None,[tib!],None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Lexer,{X} MhdWb./L.,"1,441",None,[diubde],None,s.
8,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Wilmanns Dt.Gramm.,{X} DGr./W.,"2,343",None,[diubde],None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Grimm DWB²,{X} DWB,"6,9296,58",None,[Diebde],None,None


In [29]:
df_lemmas.columns

Index(['Bundesland', 'Bundesland_idno', 'Gemeinde', 'Gemeinde_idno',
       'Großregion', 'Großregion_idno', 'Kleinregion', 'Kleinregion_idno',
       'Ort', 'Ort_idno', 'archiv', 'bibl', 'certainty', 'cit_def',
       'cit_interp', 'cit_note', 'cit_pRef', 'cit_quote', 'cit_re', 'cit_ref',
       'cit_type', 'date', 'entry_id', 'etym', 'filename', 'fragebogenNummer',
       'hauptlemma_normalized', 'hauptlemma_orig', 'hauptlemma_unique',
       'nebenlemma_normalized', 'nebenlemma_orig', 'nebenlemma_unique',
       'note_anmerkung', 'note_diverse', 'note_notabene', 'orig_placeName',
       'paragraph', 'placeName_id', 'pos', 'quelle', 'quelleBearbeitet',
       'seite', 'simple_placeName', 'tustep', 'usg_corresp', 'verweise'],
      dtype='object')

In [30]:
regions_info = ['Bundesland', 
                'Bundesland_idno', 
                'Gemeinde', 
                'Gemeinde_idno',
                'Großregion', 
                'Großregion_idno', 
                'Kleinregion', 
                'Kleinregion_idno',
                'Ort', 
                'Ort_idno',
                'simple_placeName',]

lemmas_info = ['hauptlemma_normalized', 
               'hauptlemma_orig', 
               'hauptlemma_unique',
               'nebenlemma_normalized',
               'nebenlemma_orig', 
               'nebenlemma_unique']

In [31]:
for column in df_lemmas[regions_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column Bundesland
NaN      1099610
NÖ        412316
OÖ        273038
NTir.     175406
St.       140391
Kä.        96426
Sa.        86259
STir.      72034
Bgl.       44676
OTir.      16343
Name: Bundesland, dtype: int64

unique values for column Bundesland_idno
NaN    1099610
6       412316
5       273038
1C      175406
3       140391
2        96426
4        86259
1A       72034
7        44676
1B       16343
Name: Bundesland_idno, dtype: int64

unique values for column Gemeinde
NaN                                1218052
Wien                                 71263
Mühlheim am Inn                      34694
Gmunden                              33056
Kautzen                              32858
Kufstein                             21403
Linz                                 21281
Thiersee                             20598
Poysdorf                             19640
Rangersdorf                          16430
Stainz                               16174
Traun                      

NaN        1218052
6.5          71263
5.1c07       34694
5.3b02       33056
6.3a15       32858
1C.3e06      21403
5.2e01       21281
1C.3b03      20598
6.4d18       19640
2.1b04       16430
3.3c08       16174
5.2a39       15627
6.1n02       15541
4.2c01       13364
6.2e03       12969
6.2j15       12500
1C.1m02      11318
3.1d03       11088
2.3b05       10780
5.4b13       10607
6.2e04       10276
6.1o01       10081
1C.1n01       9854
7.2b08        9769
4.5d05        9102
6.4e06        8839
6.1s06        8716
1C.3e12       8365
5.2b46        7945
1C.3i05       7827
            ...   
6.2i31          11
6.2h08          11
1C.3g02         11
1C.2a26         11
6.2b10          11
6.2b14          11
5.1f01          11
5.2b27          10
6.1j07          10
1B.0f01         10
6.2b17          10
6.2h07          10
3.2n02          10
5.2c06          10
5.4a08          10
1C.3c11         10
1C.3c10         10
7.3g09          10
5.4f12          10
7.3f01          10
6.1o04          10
6.2h27      

In [32]:
for column in df_lemmas[lemmas_info]:
    print('unique values for column {}'.format(column))
    #print(df_lemmas[column].unique())
    print(df_lemmas[column].value_counts(dropna=False))
    print()

unique values for column hauptlemma_normalized
NaN                 1101759
kemmen                 5857
sehen                  3267
lich                   3200
tuon                   3130
klein                  2633
Hôhzeit                2230
schwellen              1942
guot                   1941
schön                  1878
ein                    1854
Stein                  1846
Fleisch                1796
helfen                 1687
schwär                 1683
schauen                1611
Leib                   1441
Fuoß                   1379
stechen                1338
ferlieren              1323
schlecht               1287
laufen                 1244
Gesicht                1238
Frônleichnam           1236
Or                     1226
gruen                  1224
Eidechse               1210
Schedel                1198
Rein                   1190
Pachofen               1167
                     ...   
Schlëckerfarh             1
Weibsleüt                 1
auffrätten                1
f

NaN                    2338051
Hans                       732
hert                       703
Feter                      463
êrst                       441
Hend                       421
Schübel                    405
Kupfer                     384
trenzen                    378
êenter                     357
Före                       350
morgen                     319
dreu                       307
fërten                     305
Wurn                       284
Tübel                      261
Drumm                      258
Kuffer                     255
schienkeln                 255
Jâggel                     244
Schneck                    243
ferben                     239
Färtel                     239
Ëlexe                      238
Grängge                    238
hân                        234
Garren                     227
freusen                    220
Georgi                     220
Witiwer                    211
                        ...   
Gungger                      1
Pfarzeré

#### Building a SQLite Database

From Dataframe

In [2]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [4]:
conn = sqlite3.connect('./lemmas_db1.sqlite')

From list of dicts

In [24]:
with open('./list_lemmas.pkl', 'rb') as f:
    list_lemmas = pickle.load(f)

In [25]:
list_lemmas[0]

{'filename': './TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml',
 'entry_id': 'd154_156_qdb-d1e2',
 'date': None,
 'usg_corresp': None,
 'simple_placeName': None,
 'orig_placeName': None,
 'placeName_id': None,
 'Bundesland': None,
 'Bundesland_idno': None,
 'Großregion': None,
 'Großregion_idno': None,
 'Kleinregion': None,
 'Kleinregion_idno': None,
 'Gemeinde': None,
 'Gemeinde_idno': None,
 'Ort': None,
 'Ort_idno': None,
 'hauptlemma_unique': 'Diaun',
 'hauptlemma_orig': None,
 'hauptlemma_normalized': None,
 'nebenlemma_unique': None,
 'nebenlemma_orig': None,
 'nebenlemma_normalized': None,
 'verweise': None,
 'archiv': 'HK 154, d154^#1.1 = d1540308.pir^#1.1',
 'quelle': 'Egger Gl.',
 'quelleBearbeitet': '{1B,1C} hist.  TirWeistGl.',
 'seite': '30',
 'fragebogenNummer': None,
 'paragraph': None,
 'bibl': None,
 'tustep': ['diaun'],
 'pos': 'Subst',
 'etym': ['= rätorom. Entspr. f.', 'Dëchant', 'Dekan'],
 'note_notabene': None,
 'note_anmerkung': 'Einzelbelege aus/zu d154-d156

In [28]:
conn = sqlite3.connect('./lemmas_db.sqlite')
cur = conn.cursor()

In [30]:
q = '''
CREATE TABLE IF NOT EXISTS lemmas (
Gemeinde_idno text,
hauptlemma_orig text,
quelle text,
bibl text,
pos text,
Kleinregion_idno text,
cit_type text,
etym text,
cit_quote text,
verweise text,
Bundesland_idno text,
tustep text,
note_diverse text,
orig_placeName text,
note_anmerkung text,
nebenlemma_unique text,
Großregion text,
quelleBearbeitet text,
archiv text,
Kleinregion text,
placeName_id text,
seite text,
Gemeinde text,
Ort_idno text,
cit_note text,
Ort text,
cit_re text,
hauptlemma_unique text,
cit_interp text,
cit_def text,
Bundesland text,
date text,
nebenlemma_normalized text,
hauptlemma_normalized text,
paragraph text,
entry_id text,
nebenlemma_orig text,
certainty text,
Großregion_idno text,
cit_ref text,
fragebogenNummer text,
note_notabene text,
usg_corresp text,
filename text,
simple_placeName text,
cit_pRef text
)
'''

In [31]:
# Create table
cur.execute(q)

In [32]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('lemmas',)]


In [33]:
cur.execute('select count(*) from lemmas')
print(cur.fetchall())

[(0,)]


In [34]:
def post_row(conn, tablename, rec):
    keys = ','.join(rec.keys())
    question_marks = ','.join(list('?'*len(rec)))
    values = [str(v) for v in rec.values()]
    values = tuple(values)
    conn.execute('INSERT INTO '+tablename+' ('+keys+') VALUES ('+question_marks+')', values)

In [35]:
for idno, entry in enumerate(list_lemmas):
    post_row(conn, 'lemmas', entry)
    if idno%100000 == 0:
        print(idno)
conn.commit()
#conn.close()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
